In [ ]:
import cv2
import numpy as np
import time

# Starting webcam
vid = cv2.VideoCapture(0)

# Capturing background frame after camera warms up
background = 0
time.sleep(3)
for i in range(30):
    ret, background = vid.read()
background = np.flip(background, axis=1)  # Flipping background horizontally

while True:
    ret, img = vid.read()
    if not ret:
        break
    img = np.flip(img, axis=1)  # Flipping current frame horizontally

    # Converting BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Defining HSV ranges for detecting red color
    lower1 = np.array([0, 120, 70])
    upper1 = np.array([10, 255, 255])
    lower2 = np.array([170, 120, 70])
    upper2 = np.array([180, 255, 255])

    # Creating masks for red color ranges
    mask1 = cv2.inRange(hsv, lower1, upper1)
    mask2 = cv2.inRange(hsv, lower2, upper2)
    mask = mask1 + mask2

    # Removing noise using morphological opening
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=2)

    # Expanding the masked area slightly
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=1)

    # Inverting the mask
    inverse_mask = cv2.bitwise_not(mask)

    # Extracting cloak area from background
    cloaked_area = cv2.bitwise_and(background, background, mask=mask)

    # Extracting non-cloak area from current frame
    non_cloaked_area = cv2.bitwise_and(img, img, mask=inverse_mask)

    # Combining cloak area with non-cloak area
    final_output = cv2.addWeighted(cloaked_area, 1, non_cloaked_area, 1, 0)

    # Displaying output frames
    cv2.imshow("Invisibility Cloak", final_output)
    cv2.imshow("Live View", img)
    cv2.imshow("Red Mask", mask)

    if cv2.waitKey(1) == ord('q'):
        break

# Releasing resources
vid.release()
cv2.destroyAllWindows()
